In [ ]:
import xarray as xr

In [ ]:
def check_zeta(ds):
    """
    da_stacked containes values of zeta < -1 and their coordinates
    """
    da = ds.zeta.isel(ocean_time=1)
    da_stacked = da.where(da<-1).stack(x=['eta_rho','xi_rho'])
    da_stacked = da_stacked[da_stacked.notnull()]
    return da_stacked

In [ ]:
def check_temp(ds):
    """
    Finds lacations (eta, xi, s) of the points with high temperature
    """
    temp_da = ds.temp.isel(ocean_time=-1)
    temp_da_stacked = temp_da.where(temp_da>12).stack(x=['eta_rho', 'xi_rho', 's_rho'])
    temp_da_stacked = temp_da_stacked[temp_da_stacked.notnull()]
    return temp_da_stacked

In [ ]:
ds = xr.open_dataset('fram_data/roho160_rst.nc')

In [ ]:
da = check_temp(ds)
len(da)

In [ ]:
bad_point = da[0]
eta, xi, s = bad_point.eta_rho.values, bad_point.xi_rho.values, bad_point.s_rho.values
print(f"Xi: {xi}; Eta: {eta}")

In [ ]:
ds.temp.isel(ocean_time=-1, eta_rho=slice(eta-5,eta+5), xi_rho=slice(xi-5,xi+5)).sel(s_rho=s).plot(figsize=(10, 5))

In [ ]:
ds.h.isel(eta_rho=slice(eta-5,eta+5), xi_rho=slice(xi-5,xi+5)).plot(figsize=(10, 5))